In [1]:

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV, StratifiedKFold
import warnings
warnings.filterwarnings('ignore')

import re
from sklearn.model_selection import train_test_split #for split the data
from sklearn.metrics import accuracy_score  #for accuracy_score
from sklearn.model_selection import KFold #for K-fold cross validation
from sklearn.model_selection import cross_val_score #score evaluation
from sklearn.model_selection import cross_val_predict #prediction
from sklearn.metrics import confusion_matrix #for confusion matrix
import matplotlib.pyplot as plt
%matplotlib inline



In [2]:
train_df=pd.read_csv('train.csv')
test_df=pd.read_csv('test.csv')

In [3]:
print(train_df.shape)
train_df.head()

(891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
print(test_df.shape)
test_df.head()

(418, 11)


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


### Preprocess

In [5]:
# Fill in all the N/A values
train_df['Embarked'].fillna(train_df['Embarked'].mode()[0], inplace = True)
test_df['Embarked'].fillna(train_df['Embarked'].mode()[0], inplace = True)

train_df['Fare'].fillna(train_df['Fare'].median(), inplace = True)
test_df['Fare'].fillna(train_df['Fare'].median(), inplace = True)

train_df['Age'].fillna(train_df['Age'].median(), inplace = True)
test_df['Age'].fillna(train_df['Age'].median(), inplace = True)


In [6]:
# Define family size
train_df['FamilySize'] = train_df['SibSp'] + train_df['Parch'] + 1 #Including themselves
test_df['FamilySize'] = test_df['SibSp'] + test_df['Parch'] + 1

# train_df['IsAlone'] = (train_df['FamilySize'] == 1).astype(int)
# test_df['IsAlone'] = (test_df['FamilySize'] == 1).astype(int)



In [7]:
# Define bins 
train_df['AgeBin'] = pd.cut(train_df['Age'], bins=[0, 12, 20, 40, 200], labels = ["Child", "Teen", "Adult", "Elder"])
test_df['AgeBin'] = pd.cut(test_df['Age'], bins=[0, 12, 20, 40, 200], labels = ["Child", "Teen", "Adult", "Elder"])

train_df['FareBin'] = pd.cut(train_df['Fare'], bins=[0,7.91,14.45,31,120], labels=['Low', 'Median', 'Average','High'])
test_df['FareBin'] = pd.cut(test_df['Fare'], bins=[0,7.91,14.45,31,120], labels=['Low', 'Median', 'Average','High'])

In [8]:
# Get the title of the person 

def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)
    if title_search:
        return title_search.group(1)
    return ""

train_df['Title'] = train_df['Name'].apply(get_title)
train_df['Title'] = train_df['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 
                                                 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

test_df['Title'] = test_df['Name'].apply(get_title)
test_df['Title'] = test_df['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 
                                                 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')


train_df['Title'] = train_df['Title'].replace('Mlle', 'Miss')
train_df['Title'] = train_df['Title'].replace('Ms', 'Miss')
train_df['Title'] = train_df['Title'].replace('Mme', 'Mrs')

test_df['Title'] = test_df['Title'].replace('Mlle', 'Miss')
test_df['Title'] = test_df['Title'].replace('Ms', 'Miss')
test_df['Title'] = test_df['Title'].replace('Mme', 'Mrs')

In [9]:
# Drop columns because too many N/A or because it's not representative or because there's preprocess
train_df.drop(['Cabin', 'Age', 'Fare', 'Name', 'Ticket', 'PassengerId'], axis=1, inplace=True)
test_df.drop(['Cabin', 'Age', 'Fare', 'Name', 'Ticket'], axis=1, inplace=True)

# Encode categorical columns
categorical_cols = ['Sex', 'Embarked', 'AgeBin', 'FareBin', 'Title']
for col in categorical_cols:
    le = LabelEncoder()
    # Fit on combined values to avoid unseen labels from test
    combined = pd.concat([train_df[col].astype(str), test_df[col].astype(str)], axis=0)
    le.fit(combined)
    train_df[col] = le.transform(train_df[col].astype(str))
    test_df[col] = le.transform(test_df[col].astype(str))

In [10]:
train_df.head()

,Survived,Pclass,Sex,SibSp,Parch,Embarked,FamilySize,AgeBin,FareBin,Title
0,0,3,1,1,0,2,2,0,2,2
1,1,1,0,1,0,0,2,0,1,3
2,1,3,0,0,0,2,1,0,3,1
3,1,1,0,1,0,2,2,0,1,3
4,0,3,1,0,0,2,1,0,3,2


In [11]:
test_df.head()

,PassengerId,Pclass,Sex,SibSp,Parch,Embarked,FamilySize,AgeBin,FareBin,Title
0,892,3,1,0,0,1,1,0,2,2
1,893,3,0,1,0,2,2,2,2,3
2,894,2,1,0,0,1,1,2,3,2
3,895,3,1,0,0,2,1,0,3,2
4,896,3,0,1,1,2,3,0,3,3


### Random Forest

In [12]:
# Features/target
y = train_df['Survived']
X = train_df.drop('Survived', axis=1)

# Holdout split (keep stratification)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# Base model
rf = RandomForestClassifier(random_state=42, n_jobs=-1)

# Hyperparameter grid (smaller trees, and class_weight biasing toward death class 0)
# param_grid = {
#     'n_estimators': [600],
#     'max_depth': [3, 4, 5],
#     'max_leaf_nodes': [8, 16, 32],
#     'min_samples_leaf': [2, 3, 5],
#     'max_features': ['sqrt', 'log2', 0.5],
#     'class_weight': [
#         # {0: 1.0, 1: 1.0},
#         {0: 1.5, 1: 1.0}
#         # {0: 2.0, 1: 1.0},
#     ],
# }

param_grid = {
    'n_estimators': [600],
    'max_depth': [3],
    'max_leaf_nodes': [32],
    'min_samples_leaf': [2],
    'max_features': [0.5],
    'class_weight': [
        # {0: 1.0, 1: 1.0},
        # {0: 1.5, 1: 1.0},
        {0: 2.0, 1: 1.0},
    ],
}


# Stratified CV
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Grid search
grid = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    scoring='accuracy',
    cv=cv,
    n_jobs=-1,
    verbose=0,
)

grid.fit(X_train, y_train)
best_rf = grid.best_estimator_

# Training accuracy (best model)
rf_train_pred = best_rf.predict(X_train)
print('RF  training accuracy:', round(accuracy_score(y_train, rf_train_pred)*100, 2))

# Validation (holdout) accuracy (best model)
rf_val_pred = best_rf.predict(X_test)
print('RF  validation accuracy:', round(accuracy_score(y_test, rf_val_pred)*100, 2))

# Show chosen parameters
print('Best params:', grid.best_params_)


RF  training accuracy: 82.99
RF  validation accuracy: 79.85
Best params: {'class_weight': {0: 2.0, 1: 1.0}, 'max_depth': 3, 'max_features': 0.5, 'max_leaf_nodes': 32, 'min_samples_leaf': 2, 'n_estimators': 600}


In [13]:
# Retrain best model on all data
best_rf.fit(X, y)
best_rf

RandomForestClassifier(class_weight={0: 2.0, 1: 1.0}, max_depth=3,
                       max_features=0.5, max_leaf_nodes=32, min_samples_leaf=2,
                       n_estimators=600, n_jobs=-1, random_state=42)

In [14]:
# Predict on test_df and write titanic_predict.csv
raw_test = pd.read_csv('test.csv')
passenger_ids = raw_test['PassengerId']

# Ensure columns align with training features
test_features = test_df.copy()
test_features.drop(['PassengerId'], axis=1, inplace=True)
preds = best_rf.predict(test_features)

submission = pd.DataFrame({'PassengerId': passenger_ids, 'Survived': preds})
submission.to_csv('titanic_predict.csv', index=False)
print('Wrote titanic_predict.csv with', len(submission), 'rows')


Wrote titanic_predict.csv with 418 rows
